<small><i>This notebook is an English version (2020) of "*Normalización de Textos con Python.ipynb*" on the collection [nlp_pydata2018 on GitHub](https://github.com/sorice/nlp_pydata2018/).</i></small>

# Text Normalization with Python

La **Normalización de Textos** *(text normalization)*: es el subproceso que implica mezclar
diferentes formas de escritura en una sola apropiada y aceptable; por ejemplo un 
documento puede contener los símbolos “Señor”, “señor”, “Sr.”,”Sr” todos ellos 
deben ser normalizados a una única forma.[[1](#Indurkhya2008)]

**Tips**:

* El signo más importante es el **punto final**. (Abel2015)
* El segundo signo más importante es el **underscore** o "_". Este permite marcar 
[collocations](#collocations) para el posterior procesamiento del texto.
* Un espacio en blanco antes y después de cada punto final descomplejiza las expresiones regulares.
(Abel2015)

**Preparando el ambiente para el pre-procesamiento.**

In [1]:
import re
import string

LETTERS = ''.join([string.ascii_letters, string.digits])

## Punctuation Signs

### Signs out of ASCII & Latin1 range

This an example of rare ASCII quotation marks which usually appears in reach texts. This function filter those quotations to avoid rare characters.

In [2]:
def punctuation_filter(text):
   text = re.sub(
                 u'(?:\xc2|\xa0)|'
                 u'(?:\\xe2\\x80\\x9d|\\xe2\\x80\\x9c)|'       #Del “” en ascii.
                 u'(?:\u201c|\u201d)|'                         #Del “” en utf8.
                 u'(?:["]|[\'])'                               #Del comillas dobles y simples sin decodificar.
                 ,' ',text)
   text = re.sub(u'(?:\\xe2\\x80\\x99|\\xe2\\x80\\x98)|'       # Del ‘’ en ascii.
                 u'(?:\u2018|\u2019)'                          # Del ‘’ en ascii
                 ,'\'',text) 
   text = re.sub(u'(?:\\xe2\\x80\\x93)|'                       # Elimina guion largo ó – en ascii.
                 u'(?:\u2013)'                                 #Guión largo codificación utf8.
                 ,' - ',text)
   return text

**Note**: This func it is only a small example, a more developed func for this it is included in :func:preprocess.punctuation. The must important detail is that text without this errors cleaned will raise some error in the rest of normalization pipeline.

### Los 3 puntos seguidos ...

Para el análisis semántico algo importante son los **puntos finales** de oración. Sin embargo
para el tratamiento con expresiones regulares los tres puntos es un signo muy complejo.
Aunque aún no está claro cuál sería el patrón por el cual sustituirlo con el siguiente código
se eliminan.

**Nota**: este fue un código problemático, hice 3 implementaciones hasta que esta funcionó 
correctamente.

In [3]:
def del_contiguous_point_support(text):
   for i in re.finditer('[.]\s*?[.]+?[\s|[.]]*',text):
      for j in range(i.start(),i.end()):
         if text[j] == '.' or text[j]==' ':
            text = text[:j]+' '+text[j+1:]
   return text

### Tokens Especiales

**Cambios a nivel morfológico y léxico.**

### Correos Electrónicos y Expresiones Multipalabras

Algunos tokens como los correos electrónicos **pedro@gmail.com**, o **enseñanza - aprendizaje**,
**Firefox-v0.8** deben ser mantenidas por su valor semántico ya sea como sustantivos o sintagmas
nominales.

In [5]:
def contiguos_string_recognition_support(text):
   text = re.sub('\n-','\n- ',text)
   # support for email address is inside the regexp
   for i in re.finditer('[.]\w*|-\w*|@\w*',text): 
      for j in range(i.start(),i.end()):
         if j<(len(text)-1) and text[j] in string.punctuation and text[
         j+1] not in string.whitespace:
            text = text[:j]+'_'+text[j+1:]
   return text

### URLs

<a id='urls'></a>
Otro token especial son las *URLs*.

In [6]:
def url_string_recognition_support(text):
   for i in re.finditer('www\S*(?=[.]+?\s+?)|www\S*(?=\s+?)|http\S*(?=[.]+?\s+?)'
                        +'|http\S*(?=\s+?)',text):
      for j in range(i.start(),i.end()):
         if text[j] in string.punctuation:
            text = text[:j]+'_'+text[j+1:]
   return text

En esta función se analizan dos situaciones URL seguida por espacio (**Expr.** *www\S*(?=\s+?)*),
y URL como token final(**Expr.** *www\S*(?=[.]+?\s+?)*) de una oración **Ej.**: **... www.google.com.*

**Nota**: es importante que al final de la cadena parseada(*text*) haya al menos un espacio. 
Así en el caso de:
*"text = 'www.google.com'"* las expresiones regulares tendrían que identificar que *'m'* es también
el final de la cadena. 
Esto haría más compleja la función de reconocimiento; cuando en realidad se
resuelve agregando un espacio al final de la cadena, antes de analizarla. 
Esto es muy sencillo de implementar en el flujo (ver como **Ej.** sección 
[add_text_end_dot](#add_text_end_dot)).

<a id='siglas_y_abreviaturas'></a>

### Siglas y Abreviaturas

Un tipo de token especial son las **siglas, abreviaturas y otros similares**. En este aspecto ha de necesitarse
un diccionario bien pulido, o tal vez un buen algoritmo para reconocer algunos. Sin embargo hay 
varios diccionarios, como el de libreoffice que pudieran utilizarse y mejorarse.

In [7]:
def abbrev_recognition_support(text):
   for i in re.finditer('Dr(?=[.]+?)|Ms.C(?=[.]+?)|Ph.D(?=[.]+?)|Ing(?=[.]+?)|Lic(?=[.]+?)',
                        text):
      text = text[:i.end()]+'_'+text[i.end()+1:]
   return text

**Hipótesis**: Los algoritmos para buscar una cadena en una lista o diccionario pueden ser algo más lentos
que las expresiones regulares. Esto es debido a que se necesita hacer un search sobre una estructura
de datos una vez por cada token, en las expresiones regulares se revisa y sustituye en el texto
completo una vez por cada patrón.

In [8]:
#Pendiente versión 2 con diccionario de LibreOffice o de Google Translator.
abbr = open('data/abbr').read()
abbrDict = {}
pattern = ':'
for word in abbr.split('\n'):
    abbrDict[word] = word
print (len(abbrDict))

def abbr_filter(text, dic):
    ntext = ''
    for word in text.split(' '):
        if word in dic:
            word = dic[word]
        ntext = ntext + word + '_'    
    return ntext

481


### Profiling de detección de siglas

In [9]:
from time import clock
text = '' #Construyendo un texto de prueba.
for word in abbrDict:
    text += word+' '
for n in range(2):
    text += text

print (len(text))

11484


In [10]:
print ('Expr')
start_time1=clock()
%timeit abbrev_recognition_support(text)
end_time1=clock()-start_time1
print ('Tiempo basado en expresiones regulares %.4f' %end_time1)

Expr
10000 loops, best of 3: 105 µs per loop
Tiempo basado en expresiones regulares 4.5335


In [11]:
print ('Dict')
start_time2=clock()
%timeit abbr_filter(text,abbrDict)
end_time2=clock()-start_time2
print ('Tiempo basado en uso de diccionarios %.4f' %end_time2)

Dict
1000 loops, best of 3: 1.07 ms per loop
Tiempo basado en uso de diccionarios 4.5355


### Profiling Result

Efectivamente la búsqueda de siglas basada en diccionarios es 10 veces más lenta que basada en
expresiones regulares, evaluado en un contexto de más de 11000 términos, lo cual equivale al tamaño
de un libro promedio.

### Palabras vacías

Aunque las palabras vacías son en esencia tokens sin significado dentro de la oración, y que actúan
generalmente como conectores, los separamos por su importancia en el PLN. Fundamentalmente en el
análisis de la eficiencia computacional y la eficiencia de los resultados de la similaridad.

In [13]:
def del_char_len_one(text):
   text = re.sub('\s\w\s',' ',text)
   return text

## Structural Normalization

El siguiente código solo marca el texto con un punto al final de la última oración, para evitar
dificultades a la hora de reconocer todas las oraciones.

### add_text_end_dot

In [13]:
def add_text_end_dot(text):
   end = len(text)-1
   i = 0
   while text[end] not in LETTERS:
      end-=1
      if text[end] == '.':
         text = text[0:end]
         i+=1
   # si ningún caracter del final antes de letras o números es punto, ents suma un '.'
   if i==0: 
      text += '.' 
   return text

## Normalization Pipeline

This process could be different depending in which is your goal at the end, the target your final data is designed.

In [15]:
import time
from nltk.tokenize import RegexpTokenizer, WordPunctTokenizer
from preprocess.punctuation import Replacer
from preprocess.data import tnlp1_path

inita = time.time()
doc_name = tnlp1_path()[:-4]
with open(doc_name+'.txt','r') as text:
    print('---------')
    #Contar los términos únicos
    tokenizer = RegexpTokenizer("\s+", gaps=True)
    tokensa = tokenizer.tokenize(text)
    tokens_uniqueA = set(tokensa)

#-------------------Special tokens recognition and normalization
initg = time.time()

with open(doc_name+'.txt','r') as text:
    print ('processing urls')
    text = url_string_recognition_support(text)
    print ('processing some special punctuation signs')
    text = punctuation_filter(text)
    print ('clean contiguous dots')
    text = del_contiguous_point_support(text)
    print ('abbrev recognition and normalization')
    #~ text = abbrev_recognition_support(text)
    print ('contiguous string recognition')
    # Esta demora mucho, hay que ver porque
    text = contiguos_string_recognition_support(text) 

with open(doc_name+'1_normalized_tokens.txt', 'w') as txt:
    txt.write(text)

#-------------------Clean all punctuation sign
print ('- Limpiando los signos de puntuación.')
text = open('test/2.3/out_'+doc_name+'1_normalized_tokens.txt','r').read()
replacer = Replacer()
chunk = replacer.replace(text)

texto = open('test/2.3/out_'+doc_name+'2_tokens_including_points.txt','w')
texto.write(chunk)
texto.close()

text = open('test/2.3/out_'+doc_name+'2_tokens_including_points.txt','r').read()
tokenizer = RegexpTokenizer("\s+", gaps=True)
tokens = tokenizer.tokenize(text)

#Contando los términos únicos
tokens_uniqueD = set(tokens)

timeg = time.time() - initg

print ('-----LIMPIEZA-------------: ', timeg)
print ('El tipo de datos de tokens es:', type(tokens))
print ("La cantidad de tokens después de limpiar es: ", len(tokens),
"\nEliminados "+str(len(tokens)-len(tokensa))+" tokens durante la limpieza.",
"\n Eliminados únicos: ", len(tokens_uniqueD)-len(tokens_uniqueA))

text = open('test/2.3/out_'+doc_name+'2_tokens_including_points.txt', 'r').read()
text = add_text_end_dot(text)

texto = open('test/2.3/out_'+doc_name+'6_clean_punctuation.txt', 'w')
texto.write(text)
texto.close()

timefa = time.time() - inita
print ('La cantidad de términos únicos al filtrar es: ', len(tokens_uniqueD))

print ('Finalizado en ', timefa)
print (time.ctime())

---------
processing urls
processing some special punctuation signs
clean contiguous dots
abbrev recognition and normalization
contiguous string recognition
- Limpiando los signos de puntuación.
-----LIMPIEZA-------------:  0.01045370101928711
El tipo de datos de tokens es: <class 'list'>
La cantidad de tokens después de limpiar es:  886 
Eliminados 42 tokens durante la limpieza. 
 Eliminados únicos:  -28
La cantidad de términos únicos al filtrar es:  346
Finalizado en  0.01274251937866211
Fri Sep  2 14:47:59 2016


## Result Analysis

Veamos que tal el resultado, versus el resultado hecho por un ser humano.

In [16]:
textout = open('test/2.3/out_'+doc_name+'6_clean_punctuation.txt').read()
texthuman = open('test/2.3/'+doc_name+'_human_analysis.txt').read()
lineout = []
linehuman=[]

for line in textout.split('.'):
   lineout.append(line)
for line in texthuman.split('.'):
   linehuman.append(line)
    
for i in range(15):#max(len(lineout),len(linehuman))):
   if i < len(lineout):
        print (lineout[i])
   if i < len(linehuman):
        print (linehuman[i])
   print  ('-----')

ACID 
ACID 
-----
 En bases de datos se denomina ACID a un conjunto de características necesarias para que una serie de instrucciones puedan ser consideradas como una transacción 
 
En bases de datos se denomina ACID a un conjunto de características necesarias para que una serie de instrucciones puedan ser consideradas como una transacción 
-----
 Así pues si un sistema de gestión de bases de datos es ACID compliant quiere decir que el mismo cuenta con las funcionalidades necesarias para que sus transacciones tengan las características ACID 
 Así pues, si un sistema de gestión de bases de datos es ACID compliant quiere decir que el mismo cuenta con las funcionalidades necesarias para que sus transacciones tengan las características ACID
-----
 En concreto ACID es un acrónimo de Atomicity Consistency Isolation and Durability 


En concreto ACID es un acrónimo de Atomicity, Consistency, Isolation and Durability
-----
 Atomicidad Consistencia Aislamiento y Durabilidad en español 
 Atomici

## Ejercicios

* **Ejercicio 1**: Sobre el filtrado y normalización de siglas, 
implemente una solución basada en heurísticas más eficiente que las mostradas en este material.
* **Ejercicio 2**: Sobre el filtrado de tokens especiales, implemente una solución
que reconozca formatos de fecha.
* **Ejercicio 3**: Encuentre otros signos de puntuación fuera del rango ascii y Latin1, e implemente 
las expresiones regulares que eviten problemas en la codificación iso8859-1 para español.
* **Ejercicio 4**: Implemente una solución con NLTK para el filtrado de stopwords.
* **Ejercicio 5**: Cambie el orden de algunas de las codificaciones en el flujo de pre-procesamiento
y vea que sucede para textos en idioma español.
* **Ejercicio 6**: Implemente el filtrado de fechas.
* **Ejercicio 7**: Ver en el notebook de NLP una función más amplia utilizando NLTK de como eliminar stop words o palabras vacías de más de un caracter.

## References

<a id='Indurkhya2008'></a>
[1] *[Indurkhya2008]* Nitin Indurkhya. Book **Handbook of Natural Language Processing**. 2008. 
p. 10 **ISBN**: 978-1-4200-8593-8

## Alphabetic Index

**Collocations**: secuencia de palabras que aparecen juntas de forma frecuente, 
estableciéndose como nuevos códigos de la lengua. Ej. “caballero negro”, “vino blanco”, 
“Estados Unidos de Norteamérica”, etc.